In [ ]:
def inception_module(x,filters_a,
                     filters_b1,filters_b2,
                     filters_c1,filters_c2,
                     filters_d1,filters_d2,
                     filters_pool_proj,name=None):
  conv_a = Conv2D(filters_a, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv_b = Conv2D(filters_b1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv_b = Conv2D(filters_b2, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_b)
  conv_c = Conv2D(filters_c1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv_c = Conv2D(filters_c2, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_c)
  conv_d = Conv2D(filters_d1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv_d = Conv2D(filters_d2, (7, 7), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_d)
  pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
  pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)
  output = concatenate([conv_a, conv_b, conv_c, conv_d, pool_proj], axis=3, name=name)
  return output

kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

In [ ]:
inp_rgb = Input(shape=(224, 224, 3))
x_rgb = Conv2D(32, (3, 3), padding='same', strides=(1, 1), activation='relu')(inp_rgb)
x_rgb = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x_rgb)
x_rgb = inception_module(x_rgb,filters_a=32,
                         filters_b1=32,filters_b2=48,
                         filters_c1=32,filters_c2=48,
                         filters_d1=32,filters_d2=48,
                         filters_pool_proj=64)
x_rgb = inception_module(x_rgb,filters_a=32,
                         filters_b1=32,filters_b2=64,
                         filters_c1=32,filters_c2=64,
                         filters_d1=32,filters_d2=64,
                         filters_pool_proj=80)
x_rgb = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x_rgb)
x_rgb = inception_module(x_rgb,filters_a=32,
                         filters_b1=32,filters_b2=80,
                         filters_c1=32,filters_c2=80,
                         filters_d1=32,filters_d2=80,
                         filters_pool_proj=96)
x_rgb = inception_module(x_rgb,filters_a=32,
                         filters_b1=32,filters_b2=96,
                         filters_c1=32,filters_c2=96,
                         filters_d1=32,filters_d2=96,
                         filters_pool_proj=128)
x_rgb = Flatten()(x_rgb)
x_rgb = Dropout(0.5)(x_rgb)
x_rgb = Dense(128, activation='relu')(x_rgb)
x_rgb = Model(inputs=inp_rgb, outputs=x_rgb)

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [ ]:
inp_sift = Input(shape=(128,))
x_sift = Flatten()(inp_sift)
x_sift = Dense(128, activation='linear')(x_sift)
x_sift = Model(inputs=inp_sift, outputs=x_sift)

In [ ]:
x = concatenate([x_rgb.output, x_sift.output])

In [ ]:
x = Dense(100, activation='relu')(x)
output = Dense(10, activation='softmax')(x)

In [ ]:
merged_model = Model(inputs=[x_rgb.input, x_sift.input], outputs=output)

In [ ]:
merged_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-5), metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
merged_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 112, 112, 32  0           ['conv2d[0][0]']                 
                                )                                                           